In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array,load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import os
import numpy as np


In [32]:
lr=1e-4
epochs=20
batch_size=32

In [34]:
imagePaths=list(paths.list_images("dataset"))
data=[]
labels=[]
for imagePath in imagePaths:
    label=imagePath.split(os.path.sep)[-2]
    image=load_img(imagePath,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)
    labels.append(label)

data=np.array(data,dtype="float32")
labels=np.array(labels)

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [35]:
data

array([[[[-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         ...,
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.84313726, -0.8980392 , -0.77254903]],

        [[-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         ...,
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.75686276, -0.7882353 , -0.67058825]],

        [[-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         ...,
         [-0.88235295, -0.8980392 , -0.7882353 ],
         [-0.8666667 , -0.88235295, -0.77254903],
         [-0.85882354, -0.8745098 , -0.7647059 ]],

        ...,

        [[ 0.3803922 , -0.9607843 , -0

In [38]:
labels.shape

(3846,)

In [39]:
data.shape

(3846, 224, 224, 3)

In [40]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

(trainx,testx,trainy,testy) = train_test_split(data,labels,test_size=0.20,random_state=42)
aug=ImageDataGenerator(rotation_range=20,
                      zoom_range=0.15,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.15,
                      horizontal_flip=True,
                      fill_mode="nearest")

In [42]:
baseModel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

for layer in baseModel.layers:
    layer.trainable=False

In [45]:
opt=Adam(lr=lr,decay=lr/epochs)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
model.fit(trainx,trainy,
          batch_size=batch_size,
          steps_per_epoch=len(trainx)//batch_size,
          validation_data=(testx,testy),
          validation_steps=len(testx)//batch_size,
         epochs=epochs)

Epoch 1/20
96/96 [==============================] - 117s 1s/step - loss: 0.3535 - accuracy: 0.8457 - val_loss: 0.0892 - val_accuracy: 0.9766
Epoch 2/20
96/96 [==============================] - 115s 1s/step - loss: 0.0989 - accuracy: 0.9648 - val_loss: 0.0533 - val_accuracy: 0.9857
Epoch 3/20
96/96 [==============================] - 79s 822ms/step - loss: 0.0587 - accuracy: 0.9809 - val_loss: 0.0449 - val_accuracy: 0.9857
Epoch 4/20
96/96 [==============================] - 67s 701ms/step - loss: 0.0484 - accuracy: 0.9846 - val_loss: 0.0388 - val_accuracy: 0.9896
Epoch 5/20
96/96 [==============================] - 67s 702ms/step - loss: 0.0359 - accuracy: 0.9882 - val_loss: 0.0335 - val_accuracy: 0.9883
Epoch 6/20
96/96 [==============================] - 67s 703ms/step - loss: 0.0282 - accuracy: 0.9905 - val_loss: 0.0344 - val_accuracy: 0.9883
Epoch 7/20
96/96 [==============================] - 68s 707ms/step - loss: 0.0277 - accuracy: 0.9911 - val_loss: 0.0315 - val_accuracy: 0.9883
Epo

In [46]:
predict=model.predict(testx,batch_size=batch_size)
predict=np.argmax(predict,axis=1)
print(classification_report(testy.argmax(axis=1),predict,target_names=lb.classes_))
model.save("model",save_format="h5")

              precision    recall  f1-score   support

   with_mask       0.99      1.00      1.00       366
without_mask       1.00      1.00      1.00       404

    accuracy                           1.00       770
   macro avg       1.00      1.00      1.00       770
weighted avg       1.00      1.00      1.00       770



In [47]:
predict

array([1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,

In [49]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(testy,predict)
predict.shape

(770,)

In [50]:
testy.shape

(770, 2)

In [56]:
testy

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [57]:
trainx.shape

(3076, 224, 224, 3)

In [58]:
trainy.shape

(3076, 2)